In [1]:
#%load_ext autoreload
#%autoreload 2

In [46]:
from oirpy.oirreader import Oirreader
from bactinfection import segmentation, automated
import matplotlib.pyplot as plt
import skimage.io
import numpy as np

from pathlib import Path
import re
import os

import dask
import dask.distributed

In [47]:
client = dask.distributed.Client()

### Specify the main folder to analyze and the main folder where to store analyzed data

In [41]:
data_folder_base = Path('/Volumes/Samsung_T5/Oevermann/Listeria_Image_Analysis/Leticia')
save_folder_base = Path('/Volumes/Samsung_T5/Oevermann/Listeria_Image_Analysis/Results_Leticia')

### Find all directories containing oir files 

In [42]:
all_dirs = []
for dirName, subdirList, fileList in os.walk(data_folder_base):
    if len(list(Path(dirName).glob('*.oir'))) > 0:
           all_dirs.append(dirName)

In [54]:
# not used
# zoom_regexp = [[x, re.findall("\_(\d+)x[\_|\.].*", x.name)] for x in all_files]

### Go through all directories and run the full analysis on each image

Each single analysis is created as a dask.delayed object and added to a list. The full computation is effectively executed after that with ```dask.compute()```.

In [50]:
all_tasks = []
for a in all_dirs[0:2]:
    foldername = Path(a)
    save_folder = save_folder_base.joinpath(foldername.relative_to(data_folder_base))
    if not save_folder.is_dir():
        save_folder.mkdir(parents=True)
    all_files = [x for x in foldername.glob("*.oir")]
    
    for f in all_files[0:2]:
        task = dask.delayed(automated.single_image_analysis)(
        filepath=f,
        diameter=60,
        save_folder=save_folder,
        nucl_channel='DAPI',
        cell_channel='Alexa Fluor 633',
        bact_channel='Alexa Fluor 488',
        actin_channel='Alexa Fluor 633',
        bact_width=5,
        bact_len=5,
        corr_threshold=0.5,
        min_corr_vol=3,
        n_std=8
        )
        all_tasks.append(task)

In [52]:
res = dask.compute(all_tasks)